In [1]:
import requests
from bs4 import BeautifulSoup
from db import db_connect, db_close

Election DB 스키마

- area table: aid(pk), aname(지역구 명)
- party table: pid(pk), pname(정당 명)
- mp table: mid(pk), mname(국회의원 명), party_id(forign key), area_id(forign key), win(당선 여부), rank(순위)

mp table에 rank 컬럼 추가

사퇴한 경우 votes가 -이므로 vote_rate를 0으로 한다.

In [2]:
def get_urls():
    url = 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0'
    area_url = []
    
    res = requests.get(url).text
    soup = BeautifulSoup(res,'html.parser')
    crawls = soup.select('table[class="nowraplinks collapsible autocollapse navbox-inner"] tbody')[6]
    crawls = crawls.select('tr')[7]
    
    for crawl in crawls.select('li a'):
        area_url.append('https://ko.wikipedia.org'+crawl.get('href'))
    
    return area_url

In [3]:
def push_mp(area_num, table_ls, db):
    cursor = db.cursor()
    for table in table_ls:
        vote_ls = []
        area_num += 1
        candidates = table.select('tr:nth-child(n+3):nth-last-child(n+3):not(.sortbottom)')
        for candidate in candidates:
            mp_name= candidate.select('td:nth-child(3)')[0].text
            votes = candidate.select('td:nth-child(5)')[0].text.split('표')[0].replace(',','')
            vote_rate = candidate.select('td:nth-child(6)')[0].text.strip('\n').split('%')[0]
            
            if votes == '-':
                votes = 0
                vote_rate = 0

            vote_ls.append((mp_name,int(votes),vote_rate))
        
        vote_ls = sorted(vote_ls, key=lambda k: k[1], reverse=True)

        for i,v in enumerate(vote_ls):
            name = v[0]
            rank = i+1
            vote = v[2]
            sql = "UPDATE mp SET rank=%s, votes=%s WHERE area_id=%s and mname=%s;"
            cursor.execute(sql, (rank, vote, area_num, name))
            db.commit()
        
    return area_num

In [4]:
db = db_connect()

In [5]:
area_url = get_urls()
area_num = 0
for url in area_url:
    table_ls = []
    res = requests.get(url).text
    soup = BeautifulSoup(res,'html.parser')
    crawl = soup.select('.mw-parser-output .wikitable>tbody')
    for c in crawl[1:-1]:
        table_ls.append(c)
    area_num = push_mp(area_num, table_ls, db)

In [6]:
db_close(db)